In [1]:
import os
import random

import torch
import numpy as np
import torch.optim as optim
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score

from looper import Looper
from tab_net import TabNet, LinearM
from data_processor import PandasCatLoader, get_feature_sizes

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
TRAIN_PATH = '../../../data_main/credit/TRAIN.csv'
VAL_PATH = '../../../data_main/credit/VAL.csv'
train = pd.read_csv(TRAIN_PATH)#.sample(10000)
val = pd.read_csv(VAL_PATH)#.sample(3000)

val.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)
train.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)

train.fillna(train.median(), inplace=True)
val.fillna(train.median(), inplace=True)

train.head(2)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,TARGET
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,1
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0


In [3]:
params = dict()
params['TARGET'] = 'TARGET'
params['embed_scaler'] = 1.6
params['min_embed_size'] = 5
params = get_feature_sizes(train.drop(params['TARGET'], axis=1), params,th=1)

Low variance items all: 0, Objects all: 0
Low variance objects: 0, High var objects: 0


In [4]:
class LabelEnc:
    def __init__(self):
        self.mappers = dict()

    def fit(self, frame, cat_fe):
        for cat_fe in cat_fe:
            labels = frame[cat_fe].unique()
            mapper = {labels[i]:i for i in range(len(labels))}
            self.mappers[cat_fe] = mapper
            
    def transform(self, frame_orig):
        frame = frame_orig.copy()
        if len(self.mappers) == 0:
            print('Use fit first')
            return None
        
        for col in self.mappers:
            frame[col] = frame[col].map(self.mappers[col])
            frame[col].fillna(max(self.mappers[col].values())+1, inplace=True)
        return frame

    def fit_transform(self, frame, cat_fes):
        self.fit(frame, cat_fes)
        return self.transform(frame)
    

In [5]:
cat_columns = list(params['cat_embed_sizes'].keys())
label_enc = LabelEnc()
train = label_enc.fit_transform(train, cat_columns)
val = label_enc.transform(val)

Use fit first
Use fit first


In [6]:
params

{'TARGET': 'SeriousDlqin2yrs',
 'embed_scaler': 1.6,
 'min_embed_size': 5,
 'real_features_size': 3,
 'num_categoical': 7,
 'cat_embed_sizes': {'NumberOfOpenCreditLinesAndLoans': {'embedding_dim': 34,
   'num_categories': 56},
  'NumberOfTime30-59DaysPastDueNotWorse': {'embedding_dim': 9,
   'num_categories': 16},
  'NumberRealEstateLoansOrLines': {'embedding_dim': 16, 'num_categories': 27},
  'NumberOfDependents': {'embedding_dim': 7, 'num_categories': 13},
  'NumberOfTime60-89DaysPastDueNotWorse': {'embedding_dim': 7,
   'num_categories': 13},
  'NumberOfTimes90DaysLate': {'embedding_dim': 10, 'num_categories': 17},
  'age': {'embedding_dim': 53, 'num_categories': 86}},
 'real_features': {'DebtRatio',
  'MonthlyIncome',
  'RevolvingUtilizationOfUnsecuredLines'},
 'input_concat_vector_size': 139,
 'high_var_objects': set()}

In [7]:
# for col in params['real_features']:
#     if col == params['TARGET']: continue
#     mean, std = train[col].mean(), train[col].std()
#     if std == 0: continue
#     train[col] = (train[col] - mean)/std
#     val[col] = (val[col] - mean)/std

In [8]:
def make_dataset(train_frame, val_frame,params, shuffle=True, batch_size=512):
    loaders = []
    train_frame.reset_index(drop=True, inplace=True)
    val_frame.reset_index(drop=True, inplace=True)
    for cur_data in [train_frame, val_frame]:
        source = PandasCatLoader(cur_data, params)
        loader = torch.utils.data.DataLoader(
            source, batch_size=batch_size, num_workers=12, shuffle=shuffle
        )
        
        loaders.append(loader)

    return loaders

In [10]:
# train_set, val_set = make_dataset(train, val, params, shuffle=True, batch_size=128)

In [8]:
len(train_set)*128

89984

In [16]:
# data = next(iter(val_set))
# data

In [13]:
#set(data['target'].cpu().numpy())

In [14]:
# def dict_to_cuda(data, device):
#     for key in data:
#         if isinstance(data[key], dict):
#             for k in data[key]:
#                 data[key][k] = data[key][k].to(device)
#         else: data[key] = data[key].to(device)
            
            
# for key in data:
#     if isinstance(data[key], dict):
#         for k in data[key]:
#             data[key][k] = data[key][k].cuda()
#     else: data[key] = data[key].cuda()          

In [15]:
#torch.nn.functional.softmax(model(data), dim=1).detach().cpu().numpy()[:,1]

### LINEAR BASELINE

In [9]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter = 300, penalty='l2')
clf.fit(train.drop(params['TARGET'], axis=1), train[params['TARGET']])
preds = clf.predict_proba(val.drop(params['TARGET'], axis=1))

In [10]:
roc_auc_score(val[params['TARGET']], preds[:,1])

0.701150900279962

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = False
params["batch_norm"] = True
params["mid_features"] = 13
params["num_residual"] = 1
params["drop_out"] = 0
params['out_size'] = 2

#model = TabNet(params)
model = LinearM(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)

  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.5206995928744453
accuracy_score 0.09162181914822935


 10%|█         | 1/10 [00:21<03:13, 21.51s/it, loss=0.0764, roc_auc_score=0.521, accuracy_score=0.0916]

roc_auc_score 0.7185280337977089
accuracy_score 0.9327287223440303


 20%|██        | 2/10 [00:42<02:51, 21.41s/it, loss=0.0341, roc_auc_score=0.719, accuracy_score=0.933] 

...

 80%|████████  | 8/10 [02:56<00:44, 22.42s/it, loss=0.00798, roc_auc_score=0.818, accuracy_score=0.933]

roc_auc_score 0.8204221459768705
accuracy_score 0.9338249950169424


 90%|█████████ | 9/10 [03:19<00:22, 22.47s/it, loss=0.00691, roc_auc_score=0.82, accuracy_score=0.934] 

roc_auc_score 0.8212440128271592
accuracy_score 0.9326955019600026


100%|██████████| 10/10 [03:42<00:00, 22.84s/it, loss=0.00633, roc_auc_score=0.821, accuracy_score=0.933]

### LINEAR 2 Layers baseline

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = False
params["batch_norm"] = True
params["mid_features"] = 13
params["num_residual"] = 1
params["drop_out"] = 0
params['out_size'] = 2

#model = TabNet(params)
model = LinearM(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)


  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.5247795106666544
accuracy_score 0.07836688592120125



 10%|█         | 1/10 [00:19<02:56, 19.62s/it, loss=0.0695, roc_auc_score=0.525, accuracy_score=0.0784]

roc_auc_score 0.7459528976091762
accuracy_score 0.934688725001661



 20%|██        | 2/10 [00:38<02:35, 19.50s/it, loss=0.0337, roc_auc_score=0.746, accuracy_score=0.935]

...


 80%|████████  | 8/10 [02:35<00:39, 19.54s/it, loss=0.00797, roc_auc_score=0.813, accuracy_score=0.935]

roc_auc_score 0.8149512737678615
accuracy_score 0.935419573450269



 90%|█████████ | 9/10 [02:55<00:19, 19.47s/it, loss=0.00691, roc_auc_score=0.815, accuracy_score=0.935]

roc_auc_score 0.8155969584524392
accuracy_score 0.9352866919141585



100%|██████████| 10/10 [03:14<00:00, 19.52s/it, loss=0.00634, roc_auc_score=0.816, accuracy_score=0.935]

### Embedding Batch Norm

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = False
params["batch_norm"] = True
params["mid_features"] = 64
params["num_residual"] = 1
params["drop_out"] = 0.1
params['out_size'] = 2

model = TabNet(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)

  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.5068967179356156
accuracy_score 0.8607223451204464


 10%|█         | 1/10 [00:37<05:38, 37.66s/it, loss=0.205, roc_auc_score=0.507, accuracy_score=0.861]

roc_auc_score 0.8317694699811916
accuracy_score 0.9356359149371366


 20%|██        | 2/10 [01:15<05:00, 37.61s/it, loss=0.0968, roc_auc_score=0.832, accuracy_score=0.936]

...

 80%|████████  | 8/10 [05:00<01:14, 37.45s/it, loss=0.0232, roc_auc_score=0.852, accuracy_score=0.938]

roc_auc_score 0.8538137534615344
accuracy_score 0.9386151161110309


 90%|█████████ | 9/10 [05:37<00:37, 37.40s/it, loss=0.0205, roc_auc_score=0.854, accuracy_score=0.939]

roc_auc_score 0.8575806497839111
accuracy_score 0.939404382093667


100%|██████████| 10/10 [06:15<00:00, 37.58s/it, loss=0.0184, roc_auc_score=0.858, accuracy_score=0.939]

### Embedding Layer Norm

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = True
params["batch_norm"] = False
params["mid_features"] = 64
params["num_residual"] = 1
params["drop_out"] = 0.1
params['out_size'] = 2

model = TabNet(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)


  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.5110770906625539
accuracy_score 0.9123025445490623



 10%|█         | 1/10 [00:37<05:38, 37.60s/it, loss=0.204, roc_auc_score=0.511, accuracy_score=0.912]

roc_auc_score 0.8318733031829615
accuracy_score 0.9357470791600432



 20%|██        | 2/10 [01:14<04:59, 37.48s/it, loss=0.0967, roc_auc_score=0.832, accuracy_score=0.936]

...


 80%|████████  | 8/10 [05:01<01:15, 37.84s/it, loss=0.0232, roc_auc_score=0.85, accuracy_score=0.939]

roc_auc_score 0.8522871777123373
accuracy_score 0.9385817668441588



 90%|█████████ | 9/10 [05:39<00:37, 37.97s/it, loss=0.0205, roc_auc_score=0.852, accuracy_score=0.939]

roc_auc_score 0.8558965699760912
accuracy_score 0.939404382093667



100%|██████████| 10/10 [06:17<00:00, 37.97s/it, loss=0.0184, roc_auc_score=0.856, accuracy_score=0.939]

### BAIS TRUE EMBEDDING

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = True
params["batch_norm"] = False
params["mid_features"] = 64
params["num_residual"] = 1
params["drop_out"] = 0.1
params['out_size'] = 2

model = TabNet(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)

  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.4924649569235662
accuracy_score 0.07683671087297264


 10%|█         | 1/10 [00:37<05:35, 37.25s/it, loss=0.208, roc_auc_score=0.492, accuracy_score=0.0768]

roc_auc_score 0.8283840796735984
accuracy_score 0.9353246551129984


 20%|██        | 2/10 [01:14<04:57, 37.18s/it, loss=0.0968, roc_auc_score=0.828, accuracy_score=0.935]

...

 80%|████████  | 8/10 [04:57<01:14, 37.29s/it, loss=0.0232, roc_auc_score=0.848, accuracy_score=0.939]

roc_auc_score 0.8515759209217135
accuracy_score 0.9393932656713764


 90%|█████████ | 9/10 [05:34<00:37, 37.16s/it, loss=0.0205, roc_auc_score=0.852, accuracy_score=0.939]

roc_auc_score 0.8541702557773299
accuracy_score 0.9396822926509332


100%|██████████| 10/10 [06:11<00:00, 37.14s/it, loss=0.0183, roc_auc_score=0.854, accuracy_score=0.94]

### ZERO DROP OUT

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = True
params["batch_norm"] = False
params["mid_features"] = 64
params["num_residual"] = 1
params["drop_out"] = 0
params['out_size'] = 2

model = TabNet(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        train_set,
        optimizer,
        loss_fn,
        val_funcs)


  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.4924649569235662
accuracy_score 0.07683671087297264



 10%|█         | 1/10 [00:37<05:37, 37.47s/it, loss=0.205, roc_auc_score=0.492, accuracy_score=0.0768]

roc_auc_score 0.8309308754261338
accuracy_score 0.935658147781718



 20%|██        | 2/10 [01:14<04:59, 37.38s/it, loss=0.0951, roc_auc_score=0.831, accuracy_score=0.936]

...


 80%|████████  | 8/10 [04:58<01:14, 37.26s/it, loss=0.0225, roc_auc_score=0.858, accuracy_score=0.941]

roc_auc_score 0.8623890540428085
accuracy_score 0.9410273797481019



 90%|█████████ | 9/10 [05:35<00:37, 37.21s/it, loss=0.0198, roc_auc_score=0.862, accuracy_score=0.941]

roc_auc_score 0.8667146426477459
accuracy_score 0.9416498993963782



100%|██████████| 10/10 [06:13<00:00, 37.46s/it, loss=0.0176, roc_auc_score=0.867, accuracy_score=0.942]

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
looper = Looper(device, 'cls')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(44)    

params["layer_norm"] = True
params["batch_norm"] = False
params["mid_features"] = 64
params["num_residual"] = 1
params["drop_out"] = 0
params['out_size'] = 2

model = TabNet(params)
model.cuda();

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

val_funcs = [roc_auc_score, accuracy_score]

looper.train(model, 
        10,
        train_set, 
        val_set,
        optimizer,
        loss_fn,
        val_funcs)



  0%|          | 0/10 [00:00<?, ?it/s]

roc_auc_score 0.4924649569235662
accuracy_score 0.07683671087297264




  0%|          | 0/10 [00:37<?, ?it/s, loss=0.26, roc_auc_score=0.492, accuracy_score=0.0768]

 10%|█         | 1/10 [00:37<05:36, 37.44s/it, loss=0.26, roc_auc_score=0.492, accuracy_score=0.0768]

roc_auc_score 0.8083870390614725
accuracy_score 0.9337350067254355




 10%|█         | 1/10 [01:14<05:36, 37.44s/it, loss=0.0985, roc_auc_score=0.808, accuracy_score=0.934]

 20%|██        | 2/10 [01:14<04:59, 37.41s/it, loss=0.0985, roc_auc_score=0.808, accuracy_score=0.934]

...



 70%|███████   | 7/10 [05:14<01:57, 39.28s/it, loss=0.023, roc_auc_score=0.848, accuracy_score=0.937] 

 80%|████████  | 8/10 [05:14<01:17, 38.81s/it, loss=0.023, roc_auc_score=0.848, accuracy_score=0.937]

roc_auc_score 0.8506612237326445
accuracy_score 0.9377258023277788




 80%|████████  | 8/10 [05:52<01:17, 38.81s/it, loss=0.0203, roc_auc_score=0.851, accuracy_score=0.938]

 90%|█████████ | 9/10 [05:52<00:38, 38.52s/it, loss=0.0203, roc_auc_score=0.851, accuracy_score=0.938]

roc_auc_score 0.8535988351464034
accuracy_score 0.9380370621519171




 90%|█████████ | 9/10 [06:30<00:38, 38.52s/it, loss=0.0182, roc_auc_score=0.854, accuracy_score=0.938]

100%|██████████| 10/10 [06:30<00:00, 38.35s/it, loss=0.0182, roc_auc_score=0.854, accuracy_score=0.938]